# CS 20 : TensorFlow for Deep Learning Research
## Lecture 11 : Recurrent Neural Networks
Simple example for Many to One Classification (word sentiment classification) by Recurrent Neural Networks. 

### Many to One Classification by RNN
- Creating the **data pipeline** with `tf.data`
- Preprocessing word sequences (variable input sequence length) using `padding technique` by `tf.keras.preprocessing.sequence.pad_sequences`
- Using `tf.nn.embedding_lookup` for getting vector of tokens (eg. word, character)
- Creating the model as **Class**
- Reference
    - https://github.com/golbin/TensorFlow-Tutorials/blob/master/10%20-%20RNN/02%20-%20Autocomplete.py
    - https://github.com/aisolab/TF_code_examples_for_Deep_learning/blob/master/Tutorial%20of%20implementing%20Sequence%20classification%20with%20RNN%20series.ipynb


### Setup

In [1]:
from __future__ import absolute_import, division, print_function
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
import string
%matplotlib inline

print(tf.__version__)

1.12.0


### Prepare example data

In [2]:
words = ['good', 'bad', 'amazing', 'so good', 'bull shit', 'awesome']
y = [[1.,0.], [0.,1.], [1.,0.], [1., 0.],[0.,1.], [1.,0.]]

In [3]:
# Character quantization
char_space = string.ascii_lowercase 
char_space = char_space + ' ' + '*'
char_space = list(char_space)
char_space.insert(0, '<pad>')
print(char_space)

['<pad>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '*']


In [4]:
char2idx = {char : idx for idx, char in enumerate(char_space)}
print(char2idx)

{'<pad>': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, ' ': 27, '*': 28}


### padding example data

In [5]:
words = list(map(lambda word : [char2idx.get(char) for char in word],words))

max_length = 10
X_length = list(map(lambda word : len(word), words))
X_indices = pad_sequences(sequences=words, maxlen=max_length, padding='post', truncating='post')

print(X_length)
print(np.shape(X_indices))

[4, 3, 7, 7, 9, 7]
(6, 10)


### Define CharRNN class

In [6]:
class CharRNN(keras.Model):
    def __init__(self, num_classes, hidden_dim, max_length, dic):
        super(CharRNN, self).__init__()

        self.look_up = keras.layers.Embedding(input_dim=len(dic), output_dim=len(dic),
                                         trainable=False, mask_zero=True, input_length=max_length,
                                         embeddings_initializer=keras.initializers.Constant(np.eye(len(dic))))
        self.rnn_cell = keras.layers.SimpleRNN(units=hidden_dim, return_sequences=True,
                                               return_state=True)
        self.dense = keras.layers.Dense(units=num_classes, activation='softmax')
        
    def call(self, inputs):
        token_representation = self.look_up(inputs)        
        _, final_h = self.rnn_cell(token_representation)
        prob = self.dense(final_h)
        return prob

### Create a model of CharRNN

In [7]:
# hyper-parameter#
lr = .003
epochs = 10
batch_size = 2
total_step = int(np.shape(X_indices)[0] / batch_size)
print(total_step)

3


In [8]:
## create data pipeline with tf.data
tr_dataset = tf.data.Dataset.from_tensor_slices((X_indices, y))
tr_dataset = tr_dataset.shuffle(buffer_size = 20)
tr_dataset = tr_dataset.batch(batch_size = batch_size).repeat()
print(tr_dataset)

<RepeatDataset shapes: ((?, 10), (?, 2)), types: (tf.int32, tf.float32)>


In [9]:
char_rnn = CharRNN(num_classes=2, hidden_dim=16, dic=char2idx, max_length=10)

### Train model

In [10]:
char_rnn.compile(optimizer=tf.train.AdamOptimizer(learning_rate = lr),
                 loss=keras.losses.categorical_crossentropy,
                 metrics=['accuracy'])

In [11]:
char_rnn.fit(tr_dataset, epochs=10, steps_per_epoch=6//2)

Epoch 1/10
3/3 [==============================] - 0s 90ms/step - loss: 0.6606 - acc: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 5ms/step - loss: 0.5011 - acc: 0.8333
Epoch 3/10
3/3 [==============================] - 0s 7ms/step - loss: 0.4166 - acc: 0.8333
Epoch 4/10
3/3 [==============================] - 0s 6ms/step - loss: 0.3417 - acc: 0.8333
Epoch 5/10
3/3 [==============================] - 0s 6ms/step - loss: 0.2829 - acc: 0.8333
Epoch 6/10
3/3 [==============================] - 0s 7ms/step - loss: 0.2429 - acc: 0.8333
Epoch 7/10
3/3 [==============================] - 0s 7ms/step - loss: 0.2033 - acc: 1.0000
Epoch 8/10
3/3 [==============================] - 0s 6ms/step - loss: 0.1660 - acc: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 7ms/step - loss: 0.1436 - acc: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 7ms/step - loss: 0.1199 - acc: 1.0000
